# Homework #1 - The Linear Model

(This skeleton file is provided for HW#1 only.  You are expected to modify it for use in this and other homeworks for the course.)    

This is the README section for A0093910H's submission.
(For group submissions [when applicable], simply concatenate the student matric numbers in lexicographical order separated by a '-' (dash); e.g., A0000000X-A0000001Y)

### General Notes about this assignment 

The code below executes in the following order:

- load training data using pandas library
- Get the training data as numpy arrays
- Add bias to the x data, increasing its dimension from 20 to 21
- Perform logistic regression on training data, using batch/single-point selection and different values for η (0.05 and 0.005)
- Evaulate the resulting weights using out-of-sample-error


### Files included with this submission

hw1-1.ipynb
- source code and answer to the essay question


## Programming Exercise 1

In [1]:
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as pl
%matplotlib inline
# Plotting with style! 
import seaborn as sb 

# Size the plot appropriately for online display
pl.rcParams['figure.figsize'] = (12.0, 10.0)

Let's fix the random number generator first, in case we need results that are replicable.

In [2]:
nr.seed(3244)

### Load datasets
We also add a bias column of 1s for training data x

In [3]:
import pandas as pd
import math

def add_bias_column(x):
    return np.insert(x, 0, 1, axis=1)

input_dimension = 20
weight_dimension = input_dimension + 1 # add bias
training_data = pd.read_table('hw1-train.dat', delim_whitespace=True, header=None)
x_input = training_data.values[:,:input_dimension]
y_input = training_data.values[:,input_dimension]
N = len(x_input)
x_input = add_bias_column(x_input)


### LR

`learn_batch` function is used for batch selection.

`learn_single_point` function is used for deterministic single point selection.

Both use `lr` function as the underlying learning function

In [4]:
def compute_single_point_gradient(xi, yi, current_w):
    numerator = yi * xi
    denominator = 1 + math.exp(yi * current_w.dot(xi))
    return (numerator / denominator)

def compute_ave_gradient(xn, yn, current_w):
    gradient_sum = np.zeros(weight_dimension)
    for idx in range(N):
        gradient_sum += compute_single_point_gradient(xn[idx], yn[idx], current_w)
    ave_gradient = (-1 / N) * gradient_sum
    return ave_gradient

def compute_new_weight(ave_gradient, current_w, eta):
    return current_w + eta * (ave_gradient * (-1))

def lr(current_w, eta, use_single_point = False, iteration_number = 0):
    if use_single_point:
        idx = iteration_number % N
        gradient = -compute_single_point_gradient(x_input[idx], y_input[idx], current_w)
    else:
        gradient = compute_ave_gradient(x_input, y_input, current_w)
    return compute_new_weight(gradient, current_w, eta)

def compute_single_in_sample_error(xi, yi, current_w):
    return np.log(1 + math.exp((-yi) * current_w.dot(xi)))

def compute_ave_in_sample_error(xn, yn, current_w):
    err_sum = 0
    for idx in range(N):
        err_sum += compute_single_in_sample_error(xn[idx], yn[idx], current_w)
    err_ave = (1 / N) * err_sum
    return err_ave

def learn_batch(iterations, eta):
    print('batch learning with ' + str(iterations) + ' iterations and eta of ' + str(eta))
    w = np.zeros(weight_dimension)
    for x in range(iterations):
        w = lr(w, eta, False)
    return w

def learn_single_point(iterations, eta):
    print('single point learning with ' + str(iterations) + ' iterations and eta of ' + str(eta))
    w = np.zeros(weight_dimension)
    for x in range(iterations):
        w = lr(w, eta, True, x)
    return w

w1 = learn_batch(2333, 0.05)
w2 = learn_batch(2333, 0.005)
w3 = learn_single_point(2333, 0.05)
w4 = learn_single_point(2333, 0.005)

batch learning with 2333 iterations and eta of 0.05
batch learning with 2333 iterations and eta of 0.005
single point learning with 2333 iterations and eta of 0.05
single point learning with 2333 iterations and eta of 0.005


### Evaluation

In [5]:
testing_data = pd.read_table('hw1-test.dat', delim_whitespace=True, header=None)
x_test = testing_data.values[:,:input_dimension]
x_test = add_bias_column(x_test)
y_test = testing_data.values[:,input_dimension]
N_test = len(x_test)

def check_weight_against_test(weight, test_xi, test_yi):
    dot_product = weight.dot(test_xi)
    # print(dot_product)
    h_x = math.exp(dot_product) / (1 + math.exp(dot_product))
    prediction = 1 if (h_x >= 0.5) else -1
    # print(prediction == test_yi)
    return (prediction == test_yi)

def compute_out_of_sample_error(weight):
    err_sum = 0
    for idx in range(N_test):
        if(check_weight_against_test(weight, x_test[idx], y_test[idx]) != True):
            err_sum += 1
    err_ave = (1 / N_test) * err_sum
    return err_ave

def print_result(weight):
    print('weight:')
    print(weight.tolist())
    # print('in-sample error')
    # print(compute_ave_in_sample_error(x_input, y_input, weight))
    print('out-sample error:')
    print(compute_out_of_sample_error(weight))

### Result for η = 0.05 for T = 2333 (batch):

In [6]:
w1.tolist()

[-0.11619989953257741,
 -0.6230638861458145,
 0.8305469786461498,
 -1.0934973403011155,
 0.05572273780420706,
 -1.1139138777344821,
 -0.01296554708163877,
 1.1124953425644475,
 -0.8158812303486359,
 0.4309260722643113,
 1.4234615491768567,
 0.2768854305700301,
 -0.8809569714738535,
 -0.5974162096293618,
 0.8570422509145481,
 1.1536100733824346,
 1.3039896671074884,
 -1.3480710066622437,
 1.3424348786513611,
 -0.6163682044354374,
 -1.1006430680003119]

In [7]:
compute_out_of_sample_error(w1)

0.18433333333333332

### Result for η = 0.005 for T = 2333 (batch):

In [8]:
w2.tolist()

[0.0068855503106714875,
 -0.11440970104349153,
 0.1713339205118567,
 -0.2195381778091197,
 0.031431981110603205,
 -0.23767226697862284,
 0.01827846966222285,
 0.21209126258519115,
 -0.16241841680388647,
 0.08772493345745595,
 0.31631168396518783,
 0.05802731776823319,
 -0.15479095606960339,
 -0.09603835038685485,
 0.19544843544259954,
 0.2583708726720174,
 0.27707535469407557,
 -0.29100915213236694,
 0.2758915978261983,
 -0.12857103153081947,
 -0.2300694440172345]

In [9]:
compute_out_of_sample_error(w2)

0.26366666666666666

### Result for η = 0.05 for T = 2333 (single point):

In [10]:
w3.tolist()

[-0.13610500359911648,
 -0.7150913932415003,
 0.8717352150115324,
 -1.1547142455013584,
 -0.045438363710803155,
 -1.115596647678838,
 -0.05679034454126214,
 1.083705786851147,
 -0.9282516677434588,
 0.4247163020703131,
 1.4044058735614784,
 0.178292970315381,
 -0.7907827543185035,
 -0.6985461402096789,
 0.8544660529869273,
 1.1306861114378426,
 1.2962022062852565,
 -1.4799120604127372,
 1.4143478871384088,
 -0.6600702563762323,
 -1.0755395024117327]

In [11]:
compute_out_of_sample_error(w3)

0.22266666666666665

### Result for η = 0.005 for T = 2333 (single point):

In [12]:
w4.tolist()

[-0.00587569360276918,
 -0.13261359159470548,
 0.17183769594011664,
 -0.23245823928057666,
 -0.0012891321734608944,
 -0.247474908095806,
 0.004545199830573823,
 0.20677978266374542,
 -0.1799303285184058,
 0.08439129567952228,
 0.30812906316812844,
 0.034969577252142864,
 -0.14550272003141143,
 -0.10943277154337429,
 0.18726770702824713,
 0.24725378855802663,
 0.2687787882441544,
 -0.3136832011330746,
 0.27764022295336366,
 -0.1388937393317604,
 -0.22247117336152533]

In [13]:
compute_out_of_sample_error(w4)

0.19333333333333333

## Essay Questions

_You may choose to do the essay questions here in the .ipynb notebook, but you are welcomed to use a word processor instead and write your solutions there instead (and convert it into .pdf format).  If you do that, please ensure to delete this section._

1\. [LFD Exercise 1.2] Suppose that we use a perceptron to detect spam messages. Let's say that each email messages represented by the frequency of occurrence of keywords, and the output is +1 if the message is considered spam.

    1. Can you think of some keywords that will end up with a large positive weight into perceptron?

    2. How about keywords that will get a negative weight?

    3. What parameter in the perceptron directly affects how many borderline messages end up classified as spam?


1.
- "ads"
- "lottery"
- "win"

2.

- "account"
- "password"
- "booking"
- "reservation"
- "important"

3.

the bias value

2\. Consider a coin tossing experiment. You toss a coin 100 times, with the result of heads 70 times and tails 30 times. We denote the probability of heads of this coin as Θ. Now consider a coin toss.

    1. Build a model using maximum lilkelihood estimation (MLE) to infer Θ.

    2. Can we judge that this is an unfair coin? Explain your answer.

1.

Assume prior belief $= P(Θ) = 0.5$

By Bayes Theorem, probability density function of Θ given observations:

\begin{aligned}
p(Θ\vert H=70,T=30)
&={\frac {P(H=30\,\vert\,Θ,N=100)\,p(Θ)}{p(H=70,T=30)}} \\
&={\frac {{100 \choose 30}\,Θ^{70}\,(1-Θ)^{30}\times 0.5}{1}} \\
&=0.5{100 \choose 30}\,Θ^{70}\,(1-Θ)^{30}
\end{aligned}

Maximize $p(Θ\vert H=70,T=30)$ by differentiation,

\begin{aligned}
{\frac {dp(Θ\vert H=70,T=30)}{dΘ}} = 0.5{100 \choose 30}\,Θ^{69}\,(1-Θ)^{29}(100Θ-70) = 0 \\
\end{aligned}

\begin{aligned}
Θ = 0.7
\end{aligned}

2.

True probablity of head of a fair coin = 0.5

At 95% confidence level,

standard error:
\begin{aligned}
s = {\sqrt {\frac {p\,(1-p)}{n}}}\leq {\sqrt {\frac {0.5\times 0.5}{100}}}={\frac {1}{20}}
\end{aligned}
Maxium error:
\begin{aligned}
E=Zs={\frac {Z}{20}}={\frac {1.96}{20}}=0.098
\end{aligned}
Hence, for a fair coin, the probablity of having a head $Θ'$ should satisfy: 

\begin{aligned}
0.5-0.098<\;&Θ'<0.5+0.098 \\
0.402<\;&Θ'<0.598
\end{aligned}

0.7 clearly does not fall into the interval, hence we conclude that the coin is not fair at 95% confidence level.


3\. In the programming logistic regression, part (c), we did away with the stochastic idea of SGD and substituted a round-robin version, which deterministically uses the next point in turn to perform the gradient descent. Describe whether you think this is a good robust idea or not for datasets in general.

This might not be a good robust idea for datasets in general.

If the dataset is random in nature, then the round-robin version would effectively be the same as SGD. However, realworld datasets may have certain intrinsic patterns within the data. This coupled with the round-robin would tilt the training result depending on the number of iterations and the size of the data. 

For example, imagine the case where the training data has distinctive patterns for the first half and the second half of data points. The training result obtained when round-robin stopped at the middle of the data might be drastically different from the result obtained when round-robin stopped at the end of the data.

## Statement of Individual Work

Please initial (between the square brackets) one of the following statements.

[X] I, A0093910H, certify that I have followed the CS 3244 Machine Learning class guidelines for homework assignments.  In particular, I expressly vow that I have followed the Facebook rule in discussing with others in doing the assignment and did not take notes (digital or printed) from the discussions.  


### References

I have refered to the following list of people and websites in preparing my homework submission:

- Zhou Yichen
- [Pandas](http://pandas.pydata.org/)